In [15]:
pip install mlxtend


     |████████████████████████████████| 1.5MB 3.9MB/s eta 0:00:01
     |████████████████████████████████| 11.1MB 12.9MB/s eta 0:00:01
     |████████████████████████████████| 9.2MB 47.8MB/s eta 0:00:01
     |████████████████████████████████| 34.5MB 566kB/s  eta 0:00:01
     |████████████████████████████████| 307kB 41.3MB/s eta 0:00:01
     |████████████████████████████████| 12.4MB 56.2MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/98/5d/5738903efe0ecb73e51eb44feafba32bdba2081263d40c5043568ff60faf/numpy-1.24.4-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
     |████████████████████████████████| 307kB 80.6MB/s eta 0:00:01
     |████████████████████████████████| 1.2MB 62.5MB/s eta 0:00:01
     |████████████████████████████████| 112kB 70.1MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/ec/57/56b9bcc3c9c6a792fcbaf139543cee77261f3651ca9da0c93f5c1221264b/python_dateutil-2.9.0.post0-py2.py3-none-any.whl
     |██████████████████████████

Note: you may need to restart the kernel to use updated packages.


In [ ]:
#pip install refractio[snowflake]

In [2]:
from refractio import snowflake
dataset = snowflake.execute_query(f" SELECT * FROM RETAIL_DB.RETAIL_MKT_BSKT_SCHEMA.ORDER_INVENTORY")
dataset

User name picked from OS env: vikaskumar.10710602@ltimindtree.com
Fetching connections created by vikaskumar.10710602@ltimindtree.com user
Connection names fetched ['RETAIL_MKT_BSKT_CXN'], created by vikaskumar.10710602@ltimindtree.com
Connection object created: <snowflake.connector.connection.SnowflakeConnection object at 0x7f70f43be1f0>
Please close the connection after use!


,ORDER_DATE,ORDER_ID,PRODUCT_ID,PRODUCT,QUANTITY,UNIT_PRICE,SHIP_MODE,TOTAL_SALES,CUSTOMER_ID
0,1/31/2021 0:00,BU-2013-7380,10002439,shrimp,167,74.69,Standard Class,12473.23,MH-7455
1,1/31/2021 0:00,BU-2013-7380,10000353,almonds,278,15.28,Standard Class,4247.84,MH-7455
2,1/31/2021 0:00,BU-2013-7380,10000994,avocado,324,46.33,Standard Class,15010.92,MH-7455
3,1/31/2021 0:00,BU-2013-7380,10000475,vegetables mix,124,58.22,Standard Class,7219.28,MH-7455
4,1/31/2021 0:00,BU-2013-7380,10003338,green grapes,680,86.31,Standard Class,58690.80,MH-7455
...,...,...,...,...,...,...,...,...,...
72624,3/30/2023 0:00,US-2013-151225,10004883,chicken,290,86.72,Standard Class,25148.80,GK-14620
72625,3/30/2023 0:00,US-2013-151225,10001990,tropical fruit,314,56.47,Standard Class,17731.58,GK-14620
72626,3/30/2023 0:00,US-2013-151225,10000425,other vegetables,447,92.13,Standard Class,41182.11,GK-14620
72627,3/30/2023 0:00,US-2013-151225,10001410,vinegar,701,89.69,Standard Class,62872.69,GK-14620


In [9]:
import pandas as pd
product_set=dataset.groupby("ORDER_ID")["PRODUCT"].apply(set).to_dict()

In [10]:
filter_basket = {key:value for key, value in product_set.items() if len(value)>1}
List_order_id=list(filter_basket.keys())
List_product_id=[list(values) for values in filter_basket.values()]
dict1={"ORDER_ID":List_order_id,"PRODUCT":List_product_id}
df1=pd.DataFrame(dict1)
print(df1)

           ORDER_ID                                            PRODUCT
0      AE-2013-1130  [burgers, escalope, cooking oil, green tea, ch...
1      AE-2013-1530  [cooking oil, cottage cheese, chocolate, turke...
2      AE-2014-2840  [newspapers, sugar, yogurt, baking powder, fru...
3      AE-2014-4120                 [red/blush wine, other vegetables]
4      AG-2013-2840  [avocado, grated cheese, french fries, green tea]
...             ...                                                ...
12160  ZI-2014-6740  [newspapers, root vegetables, beef, tropical f...
12161  ZI-2014-6750                        [french fries, salad, eggs]
12162  ZI-2014-7160      [chicken, whole milk, other vegetables, meat]
12163  ZI-2014-9540  [canned vegetables, yogurt, whole milk, shoppi...
12164  ZI-2014-9550                            [soda, sugar, UHT-milk]

[12165 rows x 2 columns]


In [11]:
# Create a list of unique items
unique_items = sorted(set(item for sublist in df1['PRODUCT'] for item in sublist))
 
# Initialize an empty dictionary to hold binary values for each item
binary_data = {item: [] for item in unique_items}
 
# Iterate over each row in the original DataFrame
for bought_together in df1['PRODUCT']:
    # Initialize a binary row for the current transaction
    binary_row = {item: 0 for item in unique_items}
    # Set the value to 1 if the item was bought in the transaction
    for item in bought_together:
        binary_row[item] = 1
    # Append the binary row to the binary_data dictionary
    for item, value in binary_row.items():
        binary_data[item].append(value)
 
#Create a DataFrame from the dictionary
binary_df = pd.DataFrame(binary_data)
 
# Concatenate with the original DataFrame
binary_df2 = pd.concat([df1['ORDER_ID'], binary_df], axis=1)
 
# Display the binary DataFrame
print(binary_df)

       Instant food products  UHT-milk  abrasive cleaner  almonds   
0                          0         0                 0        0  \
1                          0         0                 0        0   
2                          0         0                 0        0   
3                          0         0                 0        0   
4                          0         0                 0        0   
...                      ...       ...               ...      ...   
12160                      0         0                 0        0   
12161                      0         0                 0        0   
12162                      0         0                 0        0   
12163                      0         0                 0        0   
12164                      0         1                 0        0   

       antioxydant juice  artif. sweetener  asparagus  avocado  babies food   
0                      0                 0          0        0            0  \
1            

In [12]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import pandas as pd
# Load your dataset
frequent_itemsets = apriori(binary_df, min_support=0.01, use_colnames=True)

# Generate association rules
rules = association_rules(frequent_itemsets, metric="support", min_threshold=0.01)
# Filter rules based on desired metrics
#rules
filtered_rules = pd.DataFrame(rules[(rules['confidence'] > 0.01) & (rules['lift'] > 0.05)])
#
# # Print the filtered rules
print(filtered_rules)

ModuleNotFoundError: No module named 'mlxtend'